In [2]:
import re
import pandas as pd
import os

In [ ]:
with open('')
ex1 = #import a txt file
ex2 = #import another
ex3 = #import another

In [ ]:
"""
need to create parsers for:
    betting stats
    running line stats
    trainers/owners
    dunzo
"""

In [ ]:
def bulkTxtToDF(txtDir):
    if txtDir[-1] != '/':
        txtDir += '/'
    files = os.listdir(txtDir)

    for file in files:
        address = txtDir + file
        with open(address) as txtChart:
            fullChart = txtChart.readlines()
            fullDayDF = parseFullDay(fullChart)

    ## append full day df to running df and return

In [ ]:
def parseFullDay(fullChart):
    newRaceInd = [0]
    newRaceTest = re.search('Copyright 2020 Equibase Company LLC. All Rights Reserved.', line)
    for i in range(len(fullChart)):
        if re.search(newRaceTest, fullChart[i]) is not None:
            newRaceInd.append(i)

    for i in range(len(newRaceInd) - 1):
        raceDF = parseRace(fullChart[newRaceInd[i]:newRaceInd[i+1]])

    ## append raceDF to running df and return

In [9]:
def parseRace(raceChart):

    for line in raceChart:
        if re.search('Off *at:', line) is not None:
            genInd = cnt + 2
        elif re.search('Last *Raced *Pgm', line) is not None:
            horseInd = [cnt]
        elif re.search('Fractional *Times:', line):
            horseInd.append(cnt)
            endInd = cnt
        cnt += 1

    genItems = parseGenInfo(raceChart[:genInd])

    if genItems == []:
        return []
    horseItems = parseHorseInfo(raceChart[horseInd[0]:horseInd[1]])
    endItems = parseEndInfo(raceChart[endInd:])
    
    return [genItems, horseItems, endItems]

    #make df row, append to df, and return

In [8]:
def parseGenInfo(genLines, trackRecordInd):
    
    trackName, month, day, year, raceNum = parseLine1(genLines[0])
    breed = parseLine2(genLines[1])
    if breed == 'QuarterHorse':
        return []
    distance, surface = parseLine5(genLines[trackRecordInd])
    weather, conditions = parseLine9(genLines[trackRecordInd + 4])
    startTime, startNote = parseLine10(genLines[trackRecordInd + 5])
    segment1, segment2, segment3, segment4, segment5 = parseLine11(genLines[trackRecordInd + 6])

    if segment3 == '':
        segments = 2
    elif segment4 == '':
        segments = 3
    elif segment5 == '':
        segments = 4
    else:
        segments = 5
        
    return [trackName, month, day, year, raceNum, breed, distance, surface, weather, conditions, startTime, startNote, segment1, segment2, segment3, segment4, segment5, segments]

In [ ]:
def parseLine11(line):
    fullSearch = re.search(r'PP *([A-Za-z0-9/]+) *([A-Za-z0-9/]*) *([A-Za-z0-9/]*) *([A-Za-z0-9/]*) *([A-Za-z0-9/]*) *Fin', line)
    segment1 = fullSearch.group(1)
    segment2 = fullSearch.group(2)
    segment3 = fullSearch.group(3)
    segment4 = fullSearch.group(4)
    segment5 = fullSearch.group(5)

    return [segment1, segment2, segment3, segment4, segment5]

In [ ]:
fullSearch = re.search(r'PP *([A-Za-z0-9/]+) *([A-Za-z0-9/]*) *([A-Za-z0-9/]*) *([A-Za-z0-9/]*) *([A-Za-z0-9/]*) *Fin', line)

In [ ]:
segment1 = fullSearch.group(1)
segment2 = fullSearch.group(2)
segment3 = fullSearch.group(3)
segment4 = fullSearch.group(4)
segment5 = fullSearch.group(5)

In [ ]:
if segment3 == '':
    segments = 2
elif segment4 == '':
    segments = 3
elif segment5 == '':
    segments = 4
else:
    segments = 5


In [ ]:
segment2

In [ ]:
def parseLine1(line):
    fullSearch = re.search(r'([^-]*)\-([^-]*)\-([^-]*)$', line)
    trackNameRaw = fullSearch.group(1)
    dateRaw = fullSearch.group(2)
    raceNumRaw = fullSearch.group(3)

    #track name -> abbreviated name
    trackNameFull = re.sub('[^A-Za-z]', '', trackNameRaw)
    trackName = trackLongToShort[trackNameFull]

    #date
    dateSearch = re.search(r'([A-Za-z]*)(\d),(\d\d\d\d)', dateRaw)
    monthRaw = dateSearch.group(1)
    month = monthNameToNumber[monthRaw]
    day = dateSearch.group(2)
    year = dateSearch.group(3)

    #race number
    raceNum = re.search('\d?\d', raceNumRaw).group(0)

    out = [trackName, int(month), int(day), int(year), int(raceNum)]

    return out

In [ ]:
def parseLine2(line):
    breedRaw = re.search(r'-(.*)$', line).group(1)
    breed = re.sub('[^A-Za-z]', '', breedRaw)
    return breed

In [ ]:
def parseLine5(line):
    fullSearch = re.search(r'([ A-Za-z]+)(?=Track)', line).group(0)
    fullSearch = re.sub(r'[^A-Za-z]', '', fullSearch)
    specSearch = re.match(r'(([A-Z][a-z]*)*)(?=OnThe)OnThe([A-Z][a-z]*)', fullSearch)
    distanceRaw, surface = [specSearch.group(1), specSearch.group(3)]

    out = [distanceRaw, surface]

    return out

In [ ]:
def parseLine9(line):
    fullSearch = re.search(r'Weather: *([A-Z][a-z]*) *Track: *([A-Z][a-z]*)', line)
    weather = fullSearch.group(1)
    conditions = fullSearch.group(2)

    out = [weather, conditions]

    return out

In [ ]:
def parseLine10(line):
    fullSearch = re.search(r'Off at: *(\d?\d:\d\d) *Start: *([A-Z0-9][a-z0-9 ]*)', line)
    startTime = fullSearch.group(1)
    startNote = fullSearch.group(2)

    out = [startTime, startNote]

    return out

In [ ]:
#line = 'LOSALAMIT?OS-January4,2019-Race1' #first
#line = 'MAIDENCLAIMING-Thoroughbred' #second
#line = 'Four And One Half Furlongs On The Dirt Track Record: (Valiant Pete - 49.20 - August 11, 1990)' #fifth
#line = 'Copyright 2020 Equibase Company LLC. All Rights Reserved.' #race flag
#line = '    Weather:   Clear Track:Fast' #ninth
#line = '    Off at: 7:02 Start: Good for all' #tenth
#line = '   Last Raced       Pgm HorseName(Jockey)                        WgtM/E PP 1/4        1/2       1m       11/4     Str      Fin     Odds Comments' #belmont
line = '    Last Raced            Pgm HorseName(Jockey)                                      Wgt M/E PP Start             1/4         Str         Fin       Odds Comments' #reg

In [ ]:
trackLongToShort = {}
trackShortToLong = {}
tracksDF = pd.read_csv('tracks_v02.csv', delimiter=',', header=None)
for i in range(tracksDF.shape[0]):
    trackLongToShort[tracksDF.iloc[i,1]] = tracksDF.iloc[i,0]
    trackShortToLong[tracksDF.iloc[i,0]] = tracksDF.iloc[i,1]

In [ ]:
monthNameToNumber = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}